RNN : [youtube](https://www.youtube.com/watch?v=WCUNPb-5EYI)


In [146]:
import torch
from torch import nn
import os
from pathlib import Path
from string import ascii_letters
from tqdm.auto import tqdm

In [13]:
a = torch.tensor([3,4,5])
b = torch.tensor([6,7,8])
c = torch.mul(a,b)
c

tensor([18, 28, 40])

# Building dataset

[dataset link](https://download.pytorch.org/tutorial/data.zip)


In [36]:
data_path = Path('../data')
name_path = data_path / 'names'

if name_path.is_dir():
    print(f'{name_path} exists..')
else:
    print('no such directory')


../data/names exists..


['Portuguese.txt',
 'Japanese.txt',
 'Korean.txt',
 'Chinese.txt',
 'Arabic.txt',
 'French.txt',
 'Dutch.txt',
 'Vietnamese.txt',
 'Italian.txt',
 'Greek.txt',
 'English.txt',
 'Spanish.txt',
 'Czech.txt',
 'Scottish.txt',
 'Irish.txt',
 'German.txt',
 'Polish.txt',
 'Russian.txt']

In [110]:
lang2label = {
    file_name.split(".")[0]: torch.tensor([i], dtype=torch.long)
    for i, file_name in enumerate(os.listdir(name_path))
}
lang2label

{'Portuguese': tensor([0]),
 'Japanese': tensor([1]),
 'Korean': tensor([2]),
 'Chinese': tensor([3]),
 'Arabic': tensor([4]),
 'French': tensor([5]),
 'Dutch': tensor([6]),
 'Vietnamese': tensor([7]),
 'Italian': tensor([8]),
 'Greek': tensor([9]),
 'English': tensor([10]),
 'Spanish': tensor([11]),
 'Czech': tensor([12]),
 'Scottish': tensor([13]),
 'Irish': tensor([14]),
 'German': tensor([15]),
 'Polish': tensor([16]),
 'Russian': tensor([17])}

In [85]:
len(lang2label)

18

In [138]:
num_langs = len(lang2label)


In [108]:
char2idx = {letter: i for i, letter in enumerate(ascii_letters)}
num_letters = len(char2idx); 
num_letters
char2idx

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51}

In [92]:
def name2tensor(name):
    tensor = torch.zeros(len(name), 1, num_letters)
    print(tensor.shape)
    for i , char in enumerate(name):
        tensor[i][0][char2idx[char]] = 1
    return tensor

In [98]:
name2tensor("a")

torch.Size([1, 1, 52])


tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]]])

In [111]:
tensor_names = []
target_langs = []

for file in os.listdir(name_path):
    with open(os.path.join(name_path, file)) as f:
        lang = file.split(".")[0]
        names = [line.rstrip() for line in f]
        for name in names:
            try:
                tensor_names.append(name2tensor(name))
                target_langs.append(lang2label[lang])
            except KeyError:
                pass


torch.Size([5, 1, 52])
torch.Size([11, 1, 52])
torch.Size([7, 1, 52])
torch.Size([5, 1, 52])
torch.Size([6, 1, 52])
torch.Size([7, 1, 52])
torch.Size([6, 1, 52])
torch.Size([7, 1, 52])
torch.Size([4, 1, 52])
torch.Size([6, 1, 52])
torch.Size([6, 1, 52])
torch.Size([7, 1, 52])
torch.Size([6, 1, 52])
torch.Size([6, 1, 52])
torch.Size([5, 1, 52])
torch.Size([6, 1, 52])
torch.Size([4, 1, 52])
torch.Size([6, 1, 52])
torch.Size([7, 1, 52])
torch.Size([7, 1, 52])
torch.Size([10, 1, 52])
torch.Size([6, 1, 52])
torch.Size([7, 1, 52])
torch.Size([9, 1, 52])
torch.Size([8, 1, 52])
torch.Size([8, 1, 52])
torch.Size([5, 1, 52])
torch.Size([7, 1, 52])
torch.Size([6, 1, 52])
torch.Size([7, 1, 52])
torch.Size([6, 1, 52])
torch.Size([6, 1, 52])
torch.Size([5, 1, 52])
torch.Size([7, 1, 52])
torch.Size([6, 1, 52])
torch.Size([9, 1, 52])
torch.Size([4, 1, 52])
torch.Size([7, 1, 52])
torch.Size([7, 1, 52])
torch.Size([9, 1, 52])
torch.Size([5, 1, 52])
torch.Size([4, 1, 52])
torch.Size([6, 1, 52])
torch.Siz

In [115]:
len(tensor_names), len(target_langs)

(19736, 19736)

In [136]:
split = round(0.9 * len(tensor_names))
len(tensor_names[:split]), len(tensor_names[split:]) 

(17762, 1974)

In [130]:
from sklearn.model_selection import train_test_split

train_idx, test_idx = train_test_split(
    range(len(target_langs)), 
    test_size=0.1, 
    shuffle=True, 
    stratify=target_langs
)

train_dataset = [
    (tensor_names[i], target_langs[i])
    for i in train_idx
]

test_dataset = [
    (tensor_names[i], target_langs[i])
    for i in test_idx
]

In [137]:
len(train_dataset) , len(test_dataset)


(17762, 1974)

# Building the model


In [153]:
class RNN(nn.Module):
    def __init__(self , input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.in2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.in2output = nn.Linear(input_size+hidden_size, output_size)

    def forward(self, x, hidden_state):
        combined = torch.cat((x, hidden_state), 1)
        hidden = torch.sigmoid(self.in2hidden(combined))
        output = self.in2output(combined)
        return output, hidden
    
    def init_hidden(self):
        return nn.init.kaiming_uniform_(torch.empty(1, self.hidden_size))

In [154]:
HIDDEN_SIZE = 256
lr = 0.001

model = RNN(input_size=num_letters, hidden_size=HIDDEN_SIZE, output_size=num_langs)
lossfn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# training model


In [166]:
EPOCHS = 2

for epoch in tqdm(range(EPOCHS)):

    for i , (name, label) in enumerate(train_dataset):
        hidden_state = model.init_hidden()
        for char in name:
            output, hidden_state = model(char, hidden_state)
        loss = lossfn(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch : {epoch} | Loss: {loss:.4f}')

 50%|█████     | 1/2 [00:25<00:25, 25.43s/it]

Epoch : 0 | Loss: 4.4957


100%|██████████| 2/2 [00:48<00:00, 24.21s/it]

Epoch : 1 | Loss: 4.2369


# Getting accuracy


In [167]:
num_correct = 0
num_samples = len(test_dataset)

model.eval()

with torch.no_grad():
    for name, label in test_dataset:
        hidden_state = model.init_hidden()
        for char in name:
            output, hidden_state = model(char, hidden_state)
        _, pred = torch.max(output, dim=1)
        num_correct += bool(pred == label)

print(f"Accuracy: {num_correct / num_samples * 100:.4f}%")


Accuracy: 76.3425%


# Checking on custom input


In [168]:
label2lang = {label.item(): lang for lang, label in lang2label.items()}

def rnn_predict(name):
    model.eval()
    tensor_name = name2tensor(name)
    with torch.no_grad():
        hidden_state = model.init_hidden()
        for char in tensor_name:
            output, hidden_state = model(char, hidden_state)
        _, pred = torch.max(output, dim=1)
    model.train()    
    return label2lang[pred.item()]

In [170]:
rnn_predict('toheed')

torch.Size([6, 1, 52])


'English'

In [173]:
rnn_predict('slaveya')


torch.Size([7, 1, 52])


'Russian'

In [175]:
rnn_predict('qhing')

torch.Size([5, 1, 52])


'Chinese'